<a href="https://colab.research.google.com/github/Gijslake/DDWT19/blob/master/skills_classificatie/annotatie/annotatie_set_maken.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Belangrijke info:

Dit programma gaat ervan uit dat de volgende bestanden in de drive staan:


*   Milvus in zip: /Milvus_model/Milvus.zip
*   Sentence transformer model in zip: /models/model_file_all_skills_199k_15e_2023-06-27_11:39_best/model.zip
*   Cluser model in pickle: /Osso data/clustering data/kmeans_model_all_skills.pickle
*   Resultaten in CSV: /Osso data/test data/macro_scale_results_1000_vac.csv

Om het programma te draaien:
*   Klap koppen uit
*   Run de volgende kopjes door op het run icoon (&#9658;) drukken:
    *   "Drive mounten"
    *   "Pip installaties"
    *   Hierna kunnen de rest van de kopjes gedraaid worden

Onder het kopje "Vacature(s) ophalen en indelen" kunnen een aantal variables aangepast worden. Om de data te herladen moet het knopje met &#9658; opnieuw gedraaid worden. Hiervan krijg je lijst met tabellen (of één als je maar één vacature ophaalt) met onder het tabel het aantal teksten dat met en zonder filter opgehaald worden.

Als het programma niet meer draait, dan: Runtime -> Verbinding verbreken en runtime verwijderen
Hierna alles weer opnieuw draaien zoals hierboven geschreven

# Meest voorkomende teksten opzoeken

## Milvus server inladen en starten

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Pip installaties

In [2]:
!pip install pandas
!pip install pyyaml
!pip install python-swiftclient python-keystoneclient
!pip install beautifulsoup4
!pip install lxml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.9/88.9 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 397.9/397.9 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.5/323.5 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.6/128.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.8/46.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.2/101.2 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.5/107.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 61.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 32.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for netifaces: filename=netifaces-0.11.0-cp310-cp310-linux_x86_64.whl size=35004 sha256=962d8d6617b6930d08694b9c5dd3c2eccb14c6

## Preparations

### Functies voor data ophalen en verwerken

In [3]:
import yaml

def read_secrets():
    with open("drive/MyDrive/InputData/secrets.yml", 'r') as file:
        data = yaml.safe_load(file)
        return data

secrets = read_secrets()
api_key = secrets['open-api']['key']
model = 'text-davinci-003'

In [4]:
def is_inline(item):
  # Returnt true als item tag een inline tag is
  inline_items = ["a", "abbr", "acronym", "b", "bdo", "big", "button", "cite",
                  "code", "dfn", "em", "i", "img", "input", "kbd", "label", "map",
                  "object", "output", "q", "samp", "script", "select", "small",
                  "span", "strong", "sub", "sup", "textarea", "time", "tt", "var"]
  return item.name in inline_items

In [5]:
import regex as re

def extract_text_recursive(item) -> [str]:
  pattern = r"^[^a-zA-Z0-9]+\s*"
  blocks = [""]

  # Itereer over alle block in een item
  for child in item.contents:

    # Als de name None is, gaat het om een bs4.element.NavigableString, dus een stuk tekst
    if child.name == None:
      blocks[-1] = blocks[-1] + " " + child.string.strip()

    # Als de child.name in de inline tags lijst staat, dan...
    elif is_inline(child):
      result = extract_text_recursive(child)
      # ... wordt de text behandeld als deel van het huidig block tenzij...
      if len(result) == 1:
        blocks[-1] = blocks[-1] + " " + result[0]
      # ... deze toch children blijkt te hebben, dan worden de children als aparte blokken behandeld
      else:
        blocks = blocks + result

    # Als het een block tag betreft, behandel het dan als nieuw block
    else:
      result = extract_text_recursive(child)
      # Als dit block daadwerkelijk tekst bevat, voeg het toe
      if len(result) > 0:
        blocks = blocks + result
      # Anders voeg een lege string toe om eventuele tekst uit children aan toe te voegen
      else:
        blocks.append("")

  # Verwijder lijsten met "- " en dergelijke en lege strings
  blocks = [re.sub(pattern, "", block.strip(), flags=re.S) for block in blocks if block.strip() != ""]
  return blocks

## Vacatures laden, html code verwerken, scoren en schrijven naar bestand

In [6]:
import swiftclient
import json
import swiftclient

# Read the settings from the JSON file
with open('drive/MyDrive/secrets_for_colab/osso_swift.json') as f:
    settings = json.load(f)

# Create a connection to the Swift API
conn = swiftclient.Connection(
    authurl=settings['auth_url'],
    user=settings['user_name'],
    key=settings['password'],
    os_options=settings['_os_options'],
    auth_version=settings['_auth_version']
)

# Set the container name and file name
container_name = 'testcontainer_dev'

# Get the files in the container
_, file_list = conn.get_container(container_name, prefix="matching_production/jobdigger")

In [10]:
# @title Vacature(s) ophalen en indelen
import json
import random
from google.colab import widgets
from google.colab import data_table
import csv
from bs4 import BeautifulSoup

max_amt = 3 # @param {type:"integer"}
vac_counter = 0
text_list = []

# Maak direrctory als die er niet is en zet de path variable
!mkdir -p data/tmp

results = []

for file_num, vacs_file in enumerate(file_list):
  if vac_counter > max_amt:
        break

  # Haal file op
  name = vacs_file['name']
  _, obj_contents = conn.get_object(container_name, name)

  # Schrijf resultaat naar zip file
  with open("/content/data/tmp/tmp.json.zip", 'wb') as f:
    f.write(obj_contents)

  # Unzip
  file_in = "/content/data/tmp/tmp.json.zip"
  !unzip -o $file_in -d /content/data/tmp

  # Lees JSON file
  with open("/content/data/tmp/data.json", "rb") as f:
    data = json.load(f)
    for vac_num, vac in enumerate(data):
      if vac_counter > max_amt:
        break

      # Alleen groupleaders gebruiken en vacatures die ISCO_code hebben en de drie tekstgroepen hebben
      if vac["PositionOpeningNL"]["PositionProfileNL"]["PositionDetail"]["Source"]["GroupLeader"] == "yes":
        # HTML code opvragen
        html_code = list(vac["PositionOpeningNL"]["PositionProfileNL"]["PositionDetail"]["PositionTexts"].values())[0]

        # Parse HTML en split in kleinere stukjes
        soup = BeautifulSoup(html_code, "lxml")
        texts = extract_text_recursive(soup)

        for text_num, text in enumerate(texts):
          results.append([f'F{file_num}V{vac_num}T{text_num}', text, None])

        vac_counter += 1


random.shuffle(results)

with open('/content/drive/MyDrive/Osso data/annotatie data/data_sample.csv', 'w', newline='') as csvfile:
  csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_MINIMAL)
  csvwriter.writerow(['ID', 'Text', 'Contains_skill'])

  for result in results:
    csvwriter.writerow(result)

Archive:  /content/data/tmp/tmp.json.zip
  inflating: /content/data/tmp/data.json  
